# Build a random forest model at admin level

In [1]:
import os
import sys
import argparse 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import pickle
import multiprocessing

In [2]:
print('---------------------------------------------------------------------')
print('Population data process 2/7: build random forest model at admin level')
print('---------------------------------------------------------------------')

---------------------------------------------------------------------
Population data process 2/7: build random forest model at admin level
---------------------------------------------------------------------


In [3]:
#setting up paths for files
#top_folder="/home/dohyungkim/population"
#ISO="SGP"
#year="2015"

In [30]:
# Initialize parser 
my_parser = argparse.ArgumentParser(description='initial input')
my_parser.add_argument('top_folder',metavar='top_folder',type=str,help='working folder')
my_parser.add_argument('ISO',metavar='ISO',type=str,help='3 character country iso code')
my_parser.add_argument('year',metavar='year',type=str,help='population year')
args = my_parser.parse_args()
top_folder = args.top_folder
ISO = args.ISO
year = args.year
if not os.path.isdir(top_folder):
    print('The path specified does not exist')
    sys.exit()

In [4]:
wp_path=os.path.join(top_folder,ISO,"wp_data")
pop_df = pd.read_csv(os.path.join(wp_path, ISO+"_population_2000_2020.csv"))[["GID","P_2015"]]
pop_df.columns=['ADMINID', 'ADMINPOP']

In [5]:
csv_path=os.path.join(wp_path,'csv')
covar_list=['srtm_topo_100m',
            'srtm_slope_100m',
            'viirs_100m'+'_'+year,
            'osm_dst_roadintersec_100m_2016',
            'osm_dst_road_100m_2016',
            'osm_dst_waterway_100m_2016',
            'esaccilc_dst011_100m'+'_'+year,
            'esaccilc_dst040_100m'+'_'+year,
            'esaccilc_dst130_100m'+'_'+year, 
            'esaccilc_dst140_100m'+'_'+year,
            'esaccilc_dst150_100m'+'_'+year,
            'esaccilc_dst160_100m'+'_'+year,
            'esaccilc_dst190_100m'+'_'+year,
            'esaccilc_dst200_100m'+'_'+year,
            'esaccilc_dst_water_100m_2000_2012',
            'dst_coastline_100m_2000_2020',
            'px_area_100m']

In [6]:
#file list
csv_list=[None] * len(covar_list)
for i in range(len(covar_list)):
    if i!=len(covar_list)-1:
        csv_list[i] =  os.path.join(csv_path,ISO+'_'+covar_list[i]+'_ZS_mean.csv')
    else:
        csv_list[i] =  os.path.join(csv_path,ISO+'_'+covar_list[i]+'_ZS_sum.csv')

In [7]:
df_admin = pd.read_csv(csv_list[0])
df_from_each_file = (pd.read_csv(f).iloc[: , 1] for f in csv_list[1:])
df = pd.concat(df_from_each_file, axis=1, ignore_index=False, join="inner")
df = pd.concat([df_admin,df], axis=1)

In [8]:
df.columns=['ADMINID'] +covar_list

In [9]:
df = pd.merge(pop_df,df, left_on='ADMINID', right_on=df.columns[0])

In [10]:
df=df.drop(df.index[np.where(df['ADMINPOP']==0)[0]])

In [11]:
df['POPD_PPHA']=df['ADMINPOP']/(df['px_area_100m']/10000)
df['POPD_PPHA_log'] = np.log(df['POPD_PPHA'])

In [12]:
df=df.dropna()

In [13]:
df_corrected = df[~df.isin([np.inf, -np.inf, np.nan]).any(1)]

In [14]:
X, y = df_corrected[covar_list[:-1]].values, df_corrected['POPD_PPHA_log'].values

## Grid Search Cross Validation

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
print('parameter optimization in process')

parameter optimization in process


In [17]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [None, 1, 2, 3], 
    'min_samples_split': [2, 3, 4],
    'n_estimators': [500, 700, 1000]
    
}

In [18]:
rf = RandomForestRegressor(random_state = 42)

In [19]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=5)

In [20]:
cpus=multiprocessing.cpu_count()-2

In [21]:
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = k_fold, n_jobs = cpus, verbose = 0)

In [22]:
grid_search.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verb

In [23]:
print("Random Forest best score:"+ str(grid_search.best_score_))

Random Forest best score:-0.005437874309369128


In [24]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': None,
 'min_samples_split': 2,
 'n_estimators': 500}

In [25]:
best_grid = grid_search.best_estimator_

In [26]:
rf_file = os.path.join(top_folder,ISO,'rf_model')
with open(rf_file, 'wb') as f:
    pickle.dump(best_grid, f)

In [29]:
df_info=pd.read_pickle(os.path.join(top_folder,ISO,"df_info.pkl"))
df_info["rf_accuracy"]=grid_search.best_score_
df_info.to_pickle(os.path.join(top_folder,ISO,"df_info.pkl"))